In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import BitsAndBytesConfig

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME="tiiuae/falcon-rw-1b",
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
base = prepare_model_for_kbit_training(base)
base.resize_token_embeddings(len(tokenizer))

peft_model = PeftModel.from_pretrained(base, "/content/lora_adapter")
peft_model.eval()

In [ ]:
def generate_with_disclaimer(prompt, max_new_tokens=150, temperature=0.7):
    input_text = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(input_text, return_tensors="pt").to(peft_model.device)
    gen = peft_model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=temperature)
    out = tokenizer.decode(gen[0], skip_special_tokens=True)
    if "### Response:" in out:
        response = out.split("### Response:")[-1].strip()
    else:
        response = out.strip()

    disclaimer = ("\n\n---\nThis response is for educational purposes only and is NOT medical advice. "
                  "Do NOT use for diagnosis or prescription. Consult a qualified healthcare professional.")
    return response + disclaimer

print(generate_with_disclaimer("How to clean a minor cut at home?"))


In [ ]:
import pandas as pd, shutil
df = pd.DataFrame(columns=["prompt","model_response","clarity_1_5","accuracy_1_5","safety_1_5","comments","reviewer_role","reviewer_name"])
df.to_csv("human_eval_template.csv", index=False)
print("human_eval_template.csv created.")

shutil.make_archive("lora_adapter", "zip", OUTPUT_DIR)
print("Created lora_adapter.zip")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp lora_adapter.zip /content/drive/MyDrive/
print("Copied lora_adapter.zip to Google Drive MyDrive")
